## 테이블 생성
1. **Products**: 제품 정보를 저장합니다.
2. **Customers**: 고객 정보를 저장합니다.
3. **Orders**: 주문 정보를 저장합니다.
```sql
-- Products 테이블
CREATE TABLE Products (
    productID INT AUTO_INCREMENT PRIMARY KEY,
    productName VARCHAR(255) NOT NULL,
    price DECIMAL(10, 2) NOT NULL,
    stockQuantity INT NOT NULL,
    createDate TIMESTAMP
);

-- Customers 테이블
CREATE TABLE Customers (
    customerID INT AUTO_INCREMENT PRIMARY KEY,
    customerName VARCHAR(255) NOT NULL,
    email VARCHAR(255) UNIQUE NOT NULL,
    address TEXT NOT NULL,
    createDate TIMESTAMP
);

-- Orders 테이블
CREATE TABLE Orders (
    orderID INT AUTO_INCREMENT PRIMARY KEY,
    customerID INT,
    orderDate TIMESTAMP,
    totalAmount DECIMAL(10, 2),
    FOREIGN KEY (customerID) REFERENCES Customers(customerID)
);
```

## 더미데이터 추가

In [2]:
import pymysql
from faker import Faker
import random

# Faker 객체 초기화
fake = Faker()

# 데이터베이스 연결 설정
conn = pymysql.connect(
    host='localhost',  # 데이터베이스 서버 주소
    user='root',       # 데이터베이스 사용자 이름
    password='0070',  # 데이터베이스 비밀번호
    db='airbnb',       # 데이터베이스 이름
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

# Products 테이블을 위한 더미 데이터 생성
def generate_product_data(n):
    for _ in range(n):
        product_name = fake.word().capitalize() + ' ' + fake.word().capitalize()
        price = round(random.uniform(10, 100), 2)
        stock_quantity = random.randint(10, 100)
        create_date = fake.date_time_this_year()
        yield (product_name, price, stock_quantity, create_date)

# Customers 테이블을 위한 더미 데이터 생성
def generate_customer_data(n):
    for _ in range(n):
        customer_name = fake.name()
        email = fake.email()
        address = fake.address()
        create_date = fake.date_time_this_year()
        yield (customer_name, email, address, create_date)

# Orders 테이블을 위한 더미 데이터 생성
def generate_order_data(n, customer_ids):
    for _ in range(n):
        customer_id = random.choice(customer_ids)
        order_date = fake.date_time_this_year()
        total_amount = round(random.uniform(20, 500), 2)
        yield (customer_id, order_date, total_amount)

# 데이터베이스에 데이터 삽입
with conn.cursor() as cursor:
    # Products 데이터 삽입
    products_sql = "INSERT INTO Products (productName, price, stockQuantity, createDate) VALUES (%s, %s, %s, %s)"
    for data in generate_product_data(10):
        cursor.execute(products_sql, data)
    conn.commit()

    # Customers 데이터 삽입
    customers_sql = "INSERT INTO Customers (customerName, email, address, createDate) VALUES (%s, %s, %s, %s)"
    for data in generate_customer_data(5):
        cursor.execute(customers_sql, data)
    conn.commit()

    # Orders 데이터 삽입
    # Customers 테이블에서 ID 목록을 얻어옵니다.
    cursor.execute("SELECT customerID FROM Customers")
    customer_ids = [row['customerID'] for row in cursor.fetchall()]

    orders_sql = "INSERT INTO Orders (customerID, orderDate, totalAmount) VALUES (%s, %s, %s)"
    for data in generate_order_data(15, customer_ids):
        cursor.execute(orders_sql, data)
    conn.commit()

# 데이터베이스 연결 종료
conn.close()

## 실습

In [21]:
import pymysql

conn=pymysql.connect(host='localhost',user='root',password='0070',db='airbnb',charset='utf8mb4',cursorclass=pymysql.cursors.DictCursor)

with conn.cursor() as cursor:
    # 1. 새로운 제품 추가: 'Products' 테이블에 "Python Book"이라는 이름의 제품을 29.99달러 가격으로 추가합니다.
    # sql= 'insert into Products (productName, price, stockQuantity) values (%s,%s,%s)'
    # cursor.execute(sql,('Python Book',29.99,10))
    # conn.commit()

    # 2. 고객 목록 조회: 'Customers' 테이블에서 모든 고객의 정보를 조회
    # cursor.execute('select * from Customers')
    # for customer in cursor.fetchall():
    #     print(customer)

    # 3. 제품 재고 업데이트: 제품이 주문될 때마다 'Products' 테이블의 해당 제품의 재고를 감소
    # sql='update Products set stockQuantity=stockQuantity-%s where productId=%s'
    # cursor.execute(sql,(1,1))
    # conn.commit()

    # 4. 고객별 총 주문 금액 계산: 'Orders' 테이블에서 각 고객별로 총 주문 금액을 조회
    # sql='select customerID, sum(totalAmount) as totalAmount from Orders group by customerID'
    # cursor.execute(sql)
    # for totalAmout in cursor.fetchall():
    #     print(totalAmout)

    # 5. 고객 이메일 업데이트: 고객 ID를 입력받고, 새로운 이메일 주소로 업데이트
    # id=input('고객 ID : ')
    # email=input('new email : ')
    # sql='update Customers set email=%s where customerID=%s'
    # cursor.execute(sql,(email,id))
    # conn.commit()

    # 6. 주문 취소: 주문 ID를 입력받아 해당 주문을 'Orders' 테이블에서 삭제
    # id=input('주문 ID : ')
    # sql='delete from Orders where orderID=%s'
    # cursor.execute(sql,id)
    # conn.commit()

    # 7. 특정 제품 검색: 제품 이름을 기반으로 'Products' 테이블에서 검색
    # sql='select * from Products where productName like %s'
    # cursor.execute(sql,('%Book%'))
    # for data in cursor.fetchall():
    #     print(data['productName'])

    # 8. 특정 고객의 모든 주문 조회: 고객 ID를 기반으로 그 고객의 모든 주문을 조회
    # sql='select * from Orders where customerID=%s'
    # cursor.execute(sql,1)
    # for data in cursor.fetchall():
    #     print(data)

    # 9. 가장 많이 주문한 고객 찾기: 'Orders' 테이블에서 가장 많은 주문을 한 고객을 조회
    sql='select customerID, count(*) as count from Orders group by customerID order by count desc limit 1'
    cursor.execute(sql)
    print(cursor.fetchone())

conn.close()

{'customerID': 3, 'count': 5}
